<a href="https://colab.research.google.com/github/abxda/COLMEX-ML/blob/main/Semana_05_00_COLMEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Tutorial Interactivo de K-means: Entiende y Visualiza el Algoritmo

En este tutorial aprenderás de forma interactiva cómo K-means agrupa datos y encuentra centroides utilizando Python en Google Colab.

**Objetivos:**
- Comprender el proceso iterativo de asignación y actualización de centroides.
- Visualizar la formación de clusters en datos sintéticos y reales.
- Explorar el "Método del Codo" para elegir el número óptimo de clusters.

## 🔰 Paso 1: Configurar el Entorno
Importamos las librerías esenciales para el análisis numérico y la visualización.

In [ ]:
!pip install scikit-learn --quiet
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, load_iris
plt.style.use('ggplot')

## 🎨 Paso 2: Crear Datos de Prueba
Generamos datos sintéticos con 300 puntos distribuidos en 3 grupos (clusters).
Ajusta `cluster_std` para modificar la dispersión de los puntos.


In [ ]:
X, _ = make_blobs(
    n_samples=300,      # 300 puntos
    centers=3,          # 3 clusters
    cluster_std=0.8,    # Desviación estándar de cada cluster
    random_state=10     # Semilla para reproducibilidad
)

In [ ]:
# Visualizamos los datos originales:
plt.figure(figsize=(8,6))
plt.scatter(X[:, 0], X[:, 1], s=50, color='skyblue')
plt.title("Datos Originales")
plt.xlabel("Característica 1")
plt.ylabel("Característica 2")
plt.show()

## 🤖 Paso 3: Aplicar K-means
Configuramos y entrenamos el modelo K-means con 3 clusters.
Parámetros clave:
- **init='random'**: Inicialización aleatoria de los centroides.
- **n_init=10**: Se realizan 10 ejecuciones para elegir la mejor solución.
- **max_iter=300**: Máximo número de iteraciones por ejecución.


In [ ]:
kmeans = KMeans(
    n_clusters=3,
    init='random',
    n_init=10,
    max_iter=300,
    random_state=42
)
kmeans.fit(X)

## 🔍 Paso 4: Visualizar los Resultados
Mostramos cómo K-means ha agrupado los datos y la ubicación final de los centroides.


In [ ]:
plt.figure(figsize=(8,6))

# Definimos colores para distinguir los clusters:
colores = ['#FF6B6B', '#4ECDC4', '#45B7D1']
etiquetas = kmeans.labels_

# Graficamos cada cluster:
for i in range(3):
    plt.scatter(
        X[etiquetas == i, 0],
        X[etiquetas == i, 1],
        s=50,
        color=colores[i],
        label=f'Cluster {i+1}'
    )

# Graficamos los centroides con una estrella dorada:
plt.scatter(
    kmeans.cluster_centers_[:, 0],
    kmeans.cluster_centers_[:, 1],
    s=200,
    marker='*',
    color='gold',
    edgecolor='black',
    linewidth=1,
    label='Centroides'
)

plt.title("🔍 Resultado del Clustering")
plt.xlabel("Característica 1")
plt.ylabel("Característica 2")
plt.legend()
plt.show()

## 🔄 Paso 5: Evolución de los Centroides
Visualizamos cómo se mueven los centroides a lo largo de las iteraciones.
Esta función muestra el estado del clustering tras 1, 2 y 3 iteraciones.

A grandes rasgos, el algoritmo K-means sigue este flujo:

1. **Inicialización Aleatoria de Centroides**  
   - Se eligen aleatoriamente $k$ posiciones en el espacio de los datos para ubicar los centroides iniciales.  
   - Estos centroides representan el “centro” temporal de cada uno de los $k$ grupos (clusters) que queremos encontrar.

2. **Asignación de Puntos**  
   - En cada iteración, cada dato se asigna al grupo cuyo centroide esté más cerca.  
   - En la práctica, esto se hace calculando la distancia (generalmente Euclidiana) entre el punto y cada centroide, y luego seleccionando el mínimo.

3. **Actualización de Centroides**  
   - Para cada grupo, se recalcula la posición del centroide como el promedio (media) de todos los puntos que han sido asignados a ese grupo.  

4. **Iteración y Convergencia**  
   - Con las nuevas posiciones de los centroides, se repite el proceso de asignación y actualización.  
   - A medida que avanzan las iteraciones, los centroides “migran” hacia la región donde se concentra la mayor parte de los puntos de cada grupo.  
   - Este movimiento se va haciendo más pequeño hasta que los centroides se estabilizan (o se llega al número máximo de iteraciones), indicando que los clusters están bien definidos.

---

### ¿Por qué converge tan rápido en la práctica?

- **Implementaciones Eficientes:**  
  Bibliotecas como *scikit-learn* utilizan versiones optimizadas del algoritmo (conocidas como variantes de *Lloyd’s algorithm* o *Elkan’s algorithm*). Estas implementaciones están escritas en C/C++ y aprovechan estructuras de datos eficientes para minimizar los cálculos de distancia, reduciendo el tiempo de ejecución.

- **Vectorización y Cálculo en Bloque:**  
  Muchas operaciones (como la asignación de puntos a centroides) se realizan en forma vectorizada, usando *NumPy* u otras librerías de álgebra lineal, lo que aprovecha la velocidad de operaciones a nivel de bajo nivel (SIMD, paralelismo, etc.).

- **Heurísticas de Inicialización (k-means++):**  
  Aunque el ejemplo usa inicialización aleatoria, en la práctica se suele usar *k-means++* para elegir centroides iniciales “inteligentemente”. Esto suele acelerar la convergencia y mejorar la calidad de los clusters.

- **Reducción de Cálculos Innecesarios:**  
  Algunas implementaciones evitan recalcular distancias para puntos que se sabe que no cambiarán de cluster en la siguiente iteración, acelerando así la convergencia.

En conjunto, estos factores hacen que K-means, aun siendo un algoritmo iterativo, **tienda a converger rápidamente** y a encontrar clusters útiles en un número relativamente bajo de pasos.

In [ ]:
def visualizar_iteraciones(n_iteraciones):
    plt.figure(figsize=(15, 4))

    for i in range(n_iteraciones):
        # Ejecutamos K-means con (i+1) iteraciones
        km = KMeans(n_clusters=3, max_iter=i+1, init='random', n_init=1)
        km.fit(X)

        plt.subplot(1, n_iteraciones, i+1)
        plt.scatter(X[:, 0], X[:, 1], c=km.labels_, cmap='viridis', s=30)
        plt.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
                    marker='X', s=100, c='red')
        plt.title(f'Iteración {i+1}')
    plt.tight_layout()

visualizar_iteraciones(3)
plt.show()

## 📉 Paso 6: Método del Codo
Utilizamos el método del codo para determinar el número óptimo de clusters.
Calculamos la inercia (la suma de las distancias al cuadrado entre cada punto y su centroide) para distintos valores de k.


In [ ]:
inercias = []
rang_k = range(1, 10)

for k in rang_k:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inercias.append(kmeans.inertia_)

plt.figure(figsize=(8,5))
plt.plot(rang_k, inercias, 'bo-')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo')
plt.xticks(rang_k)
plt.grid(True)
plt.show()

En el **Método del Codo**, se calcula la *inercia* (o suma de distancias al cuadrado de cada punto a su centroide) para distintos valores de $k$. El objetivo es encontrar un punto en la gráfica donde añadir más clusters deje de reducir significativamente la inercia, es decir, donde la curva “se doble” y la disminución de inercia empiece a ser menos pronunciada.

Observando la gráfica adjunta:

1. **Drástica caída al principio (de $k=1$ a $k=3$)**  
   - Cuando $k$ pasa de 1 a 2 y luego a 3, la inercia baja de forma muy notable. Esto indica que el agrupamiento está mejorando sustancialmente al aumentar el número de clusters en esas primeras etapas.

2. **Punto de Inflexión alrededor de $k=3$**  
   - Después de $k=3$, la reducción en la inercia es cada vez menor. El salto de inercia entre $k=3$ y $k=4$ es más leve que los saltos anteriores.  
   - Ese “doble” o “codo” que se aprecia en la curva alrededor de $k=3$ es la señal de que, a partir de ahí, añadir más clusters ya no aporta una mejora tan significativa en la compactación de los grupos.

3. **Interpretación Práctica**  
   - Con $k=3$, se logra un buen equilibrio entre complejidad (tener más clusters) y calidad de la agrupación (baja inercia).  
   - Si se escogen más clusters (por ejemplo, 4, 5, etc.), la inercia seguirá bajando, pero la ganancia es mucho menor comparada con el salto que obtienes al pasar de 2 a 3.  
   - En muchos casos de uso, la elección de un $k\ más alto puede significar una sobresegmentación de los datos (clusters demasiado pequeños o menos interpretables).

4. **Limitaciones**  
   - El método del codo es un **criterio heurístico**. No siempre el “codo” está perfectamente definido. Aun así, en la gráfica mostrada, el codo es bastante claro en $k=3$.  
   - Se recomienda complementar con otros métodos (como el índice de silhouette) o con un criterio basado en el contexto de los datos para validar la elección de $k$.

En resumen, **elegir $k=3$** en este caso se justifica porque la inercia desciende marcadamente hasta ese punto y, a partir de ahí, la mejora adicional es mucho más pequeña. Esto indica que **3 clusters** describen razonablemente bien la estructura de los datos sin añadir complejidad innecesaria.